# ANÁLISIS EXPLORATORIO DE DATOS (EDA)

__Variables:__ FECH, FOBDOL, PAIS, POSARA, DEPTO

1. __FECH__ - Fecha del proceso (AÑO, MES)
2. __FOBDOL__ - Valor FOB en dólares
3. __PAIS__ - Código país destino
4. __POSARA__ - Posición arancelaria
5. __DEPTO__ - Código departamento origen

>"El valor FOB en dólares muestra cuánto valen las exportaciones al momento de salir del país. Este monto incluye todos los gastos necesarios para llevar la mercancía hasta el puerto y cargarla al barco o avión, como la producción, el empaque y el transporte interno. Sin embargo, no toma en cuenta el costo del seguro ni el transporte internacional hasta el destino final. Por eso, el FOB sirve para comparar el valor de los productos exportados de forma estándar y consistente entre diferentes períodos y destinos."

In [1]:
import sys
sys.path.append('..')
from utils.paths import get_file_path

import polars as pl
import plotly.express as px

In [2]:
def estilo(fig):
    fig.update_layout(
        title_font=dict(size=24, family="Arial", color="black"),
        xaxis_tickangle=-45,
        xaxis_tickfont=dict(size=12),
        yaxis_tickfont=dict(size=12),
        plot_bgcolor="white",
        paper_bgcolor="white",
        showlegend=False,
        margin=dict(t=80, l=40, r=20, b=120),
    )
    return fig

In [3]:
df = pl.scan_csv(
    get_file_path('cleaned', 'exportaciones'),
    low_memory=True
).collect()

## Análisis específicos

### Distribución temporal

In [7]:
dis_temporal = df.group_by("AÑO").agg(pl.col("FOBDOL").sum()).sort("AÑO")

In [8]:
fig = px.line(dis_temporal,
              x='AÑO',
              y='FOBDOL',
              title='Valor total exportado por año',
              labels={"AÑO": "Año", "FOBDOL": "FOB Dólares"}
              )
fig = estilo(fig)
fig.show()

> ###### *__Finalidad:__ Visualizar cómo han evolucionado las exportaciones a lo largo del tiempo, identificando tendencias generales, crecimientos o caídas en el valor FOB anual.*

### Top 10 países destino

In [9]:
top_paises = df.group_by("PAIS").agg(pl.col("FOBDOL").sum()).sort("FOBDOL", descending=True).head(10)

In [10]:
fig = px.bar(top_paises,
             x='PAIS',
             y='FOBDOL',
             title='Top 10 países destino',
             labels={"PAIS": "País", "FOBDOL": "FOB Dólares"}
             )
fig = estilo(fig)
fig.show()

> ###### *__Finalidad:__ Identificar los principales países compradores, mostrando cuáles destinos concentran la mayor parte del valor exportado.*

### Top 10 productos

In [ ]:
top_productos = df.group_by("POSARA").agg(
).sort("FOBDOL", descending=True).head(10)
top_productos = top_productos.to_pandas()
productos = {
    2709000000: 'Petróleos crudos e hidrocarburos',
    2701120010: 'Carbón bituminoso y briquetas',
    901119000: 'Café sin tostar',
    7108120000: 'Gemas y metales preciosos',
    2710192200: 'Fuelóleos y gasóleos pesados',
    7202600000: 'Ferroaleaciones de níquel',
    2704001000: 'Coques y semicoques de hulla',
    2710192100: 'Aceites de petróleo (≥70 %)',
    803901100: 'Bananas y plátanos frescos',
    603199090: 'Flores y capullos cortados, frescos'
}

top_productos = top_productos.rename(columns={'POSARA': 'PRODUCTOS'})
top_productos['PRODUCTOS'] = top_productos['PRODUCTOS'].map(productos)

In [12]:
fig = px.bar(top_productos,
             x='PRODUCTOS',
             y='FOBDOL',
             title='Top 10 productos',
             labels={"PRODUCTOS": "Productos", "FOBDOL": "FOB Dólares"}
             )
fig = estilo(fig)
fig.show()

> ###### *__Finalidad:__ Mostrar qué productos representan el mayor valor FOB exportado, destacando aquellos que son más relevantes en la canasta exportadora.*

### Top 10 departamentos

In [13]:
top_departamentos = df.group_by("DEPTO").agg(pl.col("FOBDOL").sum()).sort("FOBDOL", descending=True).head(10)

In [18]:
fig = px.bar(top_departamentos,
             x='DEPTO',
             y='FOBDOL',
             title='Top 10 departamentos de orígen',
             labels={"DEPTO": "Departamentos", "FOBDOL": "FOB Dólares"}
             )
fig = estilo(fig)
fig.show()

>###### *__Finalidad:__ Evidenciar qué departamentos del país tienen mayor participación en el total exportado, ayudando a reconocer regiones clave en la actividad exportadora.*

### Estadísticas descriptivas

In [17]:
stats = df.select(["FOBDOL", "CANTIDAD", "PNK", "AGRENA", "FLETES"]).describe()
print(stats)

shape: (9, 6)
┌────────────┬──────────────┬───────────────┬───────────────┬───────────────┬───────────────┐
│ statistic  ┆ FOBDOL       ┆ CANTIDAD      ┆ PNK           ┆ AGRENA        ┆ FLETES        │
│ ---        ┆ ---          ┆ ---           ┆ ---           ┆ ---           ┆ ---           │
│ str        ┆ f64          ┆ f64           ┆ f64           ┆ f64           ┆ f64           │
╞════════════╪══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╡
│ count      ┆ 8.930586e6   ┆ 8.930586e6    ┆ 8.930586e6    ┆ 8.930586e6    ┆ 8.930586e6    │
│ null_count ┆ 0.0          ┆ 0.0           ┆ 0.0           ┆ 0.0           ┆ 0.0           │
│ mean       ┆ 95018.065754 ┆ 240969.402535 ┆ 268696.563529 ┆ 24787.605539  ┆ 1095.947214   │
│ std        ┆ 2.5836e6     ┆ 7.5479e6      ┆ 7.1102e6      ┆ 403229.860224 ┆ 427956.783123 │
│ min        ┆ 0.01         ┆ 0.0           ┆ 0.0           ┆ 0.0           ┆ 0.0           │
│ 25%        ┆ 581.7        ┆ 50.73         ┆ 

>###### *__Finalidad:__ Resumir las principales características numéricas de las variables analizadas (FOB, cantidad, peso, fletes), facilitando su interpretación general mediante medidas como media, desviación estándar y valores extremos.*

----